# Part 1 - Exploring Sub Wallet Management

This agent has been initialised using the multitennant flag. This means a single ACA-Py instance can be used to manage multiple subwallets. Each tennant gets their own encrypted data storage for managing their own connections, credentials and interactions etc. 

A mutli-tennant ACA-Py instance contains a base wallet that is only capable of managing the creation and deletion of subwallets. Subwallets then must authenticate to the ACA-Py agent using a JWT Token generated when the subwallet is created. A tennant agent can do all funcationality of a standard ACA-Py instance.

### Useful links

* [ACA-Py mult-tennant documentation](https://github.com/hyperledger/aries-cloudagent-python/blob/main/Multitenancy.md)
* [What is mult-tennancy](https://whatis.techtarget.com/definition/multi-tenancy)


### Tutorial Structure

1. Create a subwallet for Alice (this notebook)
2. Authenticate as Alice using the JWT_Token and configure a mediator
3. Issue Alice a Credential from an External Agent
4. Alice Issues a Credential to the External Agent

### Initialise the basewallet controller

In [ ]:
%autoawait
import time
import asyncio
import pprint

from aries_basic_controller.aries_controller import AriesAgentController
    
# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://basewallet-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, multitenant=True, api_key="password")

### Check for subwallets on the agent

This should yield an mepty result, but not error. That means we successfully asked the basewallet holder multitenant agent about subwallets.

In [ ]:
response = await agent_controller.multitenant.query_subwallets()
pp.pprint(response)


### Let's create a subwallet for Alice

Below is an example payload to achieve this.

In [ ]:
## First let's create the payload

payload = {
  "image_url": "https://aries.ca/images/sample.png",
  "key_management_mode": "managed",
  "label": "Alice",
  "wallet_dispatch_type": "default",
  "wallet_key": "MySecretKey123",
  "wallet_name": "AlicesWallet",
  "wallet_type": "indy",
  "wallet_webhook_urls": [
    "http://localhost:8022/webhooks"
  ]
}

In [ ]:
## Now, we create the wallet on the agent 
response_alice = await agent_controller.multitenant.create_subwallet(payload)
pp.pprint(response_alice)

### Let's create another wallet for Joe

Note, that here we have changed the `label` and the `wallet_name` values. The `wallet_name` has to be unique. If you were to try and create another subwallet with the same wallet name, you would receive an error, because wallet names are unique identifiers.

In [ ]:
## First let's create the payload

payload = {
  "image_url": "https://aries.ca/images/sample.png",
  "key_management_mode": "managed",
  "label": "Joe",
  "wallet_dispatch_type": "default",
  "wallet_key": "MySecretKey123",
  "wallet_name": "JoesWallet",
  "wallet_type": "indy",
  "wallet_webhook_urls": [
    "http://localhost:8022/webhooks/joe"
  ]
}

In [ ]:
## Now, we create the wallet on the agent 

response_joe = await agent_controller.multitenant.create_subwallet(payload)
pp.pprint(response_joe)

### Extract the wallet ID

In [ ]:
wallet_id_alice = response_alice['wallet_id']
wallet_id_joe = response_joe['wallet_id']

print("Alice's ID: " + wallet_id_alice)
print("Joe's ID: "  + wallet_id_joe)


### Update a single subwallet

Let's update Joe's wallet label to Joeseph via the controller

In [ ]:
request_body = {
  "image_url": "https://aries.ca/images/sample.png",
  "label": "Joeseph",
  "wallet_dispatch_type": "default",
  "wallet_webhook_urls": [
    "http://localhost:8022/webhooks"
  ]
}

response = await agent_controller.multitenant.update_subwallet_by_id(request_body, wallet_id_joe)
pp.pprint(response)

### Remove the subwallet from the agent 

We can easily use the controller to remove the subwallets.

Let's go ahaead and remove Joe's wallets from the base wallet.

In [ ]:
response_joe = await agent_controller.multitenant.remove_subwallet_by_id(wallet_id_joe)

pp.pprint(response_joe)

### Check Joe's wallet has been removed

This should now give a result only containing Alice's wallet

In [ ]:
response_all_wallets = await agent_controller.multitenant.query_subwallets()
# response_single_wallet = await agent_controller.multitenant.get_single_subwallet_by_id(wallet_id)

# print(response_single_wallet)
pp.pprint(response_all_wallets)

### Get the auth token for a  subwallet

Subwallets have unique authentication tokens that can be obtained via the controller

In [ ]:
response_alice = await agent_controller.multitenant.get_subwallet_authtoken_by_id(wallet_id_alice)

pp.pprint(response_alice)

## Store Alice's JWT for use in later tutorials

The % is a magic method to store variables from a notebook to the jupyter runtime

In [ ]:
alice_jwt = response_alice["token"]
%store alice_jwt

### Terminate the controller

Let's alos terminate the controller.

In [ ]:
response = await agent_controller.terminate()
print(response)

### Continue to [Part 2](http://localhost:8888/notebooks/Alice/Part%202%20-%20Mediation%20of%20communication%20-%20Alice.ipynb) where you will learn how to interact with the ACA-Py instance as Alice